In [1]:
!pip install flask_sqlalchemy 
!pip install flask_wtf
!pip install flask_login
!pip install flask_bcrypt

In [ ]:
from flask import Flask, render_template, url_for, session, flash, request, redirect
from datetime import datetime
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.widgets import TextArea
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
import pandas as pd
import sqlite3

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SECRET_KEY'] = 'thisisasecretkey'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    password = db.Column(db.String(80))
    notes = db.relationship('Note', backref='writer', lazy='dynamic')


class Note(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    note_body = db.Column(db.String(100))
    note_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    author = db.Column(db.String(255))
    date_created = db.Column(db.TIMESTAMP, default=datetime.utcnow, nullable=False)


class RegisterForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Password"})
    submit = SubmitField("Register")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder":  "Password"})
    submit = SubmitField ("Log In")

class NewNoteForm(FlaskForm):
    title = StringField("Title", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "Title"})
    # note_body = TextAreaField("Note Body", validators=[InputRequired(), Length(max=100)], render_kw={"placeholder":  "Note Body"})
    note_body = StringField(u'Text', widget=TextArea())
    submit = SubmitField("New Post")

@app.route('/')
def index():
    return render_template ('index.html')

@app.route('/partner')
def partner():
    return render_template ('partner.html')

@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("index"))

        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)

@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('index'))

@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html',form=form)

@app.route("/food")
def food():
    sqlalchemyObj = db.engine.execute('select * from food ORDER BY Name')
    food = []
    for i in sqlalchemyObj:
        food.append(i)
    dataset = []
    dict={}
    for i in food:
        dict['id'] = i[0]
        dict['Name'] = i[1]
        dict['Lat'] = i[2]
        dict['Lng'] = i[3]
        dict['Url'] = i[4]
        dict['Img_url'] = i[5]
        dict['Img_url2'] = i[6]
        dict['Img_url3'] = i[7]
        dict['Desc'] = i[8]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template('foodcatelog.html', dataset=dataset)

@app.route("/activity")
def activity():

    sqlalchemyObj = db.engine.execute('select * from activity ORDER BY Name')
    activity = []
    for i in sqlalchemyObj:
        activity.append(i)
    dataset = []
    dict={}
    for i in activity:
        dict['id'] = i[0]
        dict['name'] = i[1]
        dict['Lat'] = i[2]
        dict['Lng'] = i[3]
        dict['url'] = i[4]
        dict['img_url'] = i[5]
        dict['desc'] = i[6]
        dict['details'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template('activitycatelog.html', dataset=dataset)

@app.route('/forum', methods=['GET','POST'])
def note():
    sqlalchemyObj = db.engine.execute('select note.title, note.note_body, note.date_created,author from Note ORDER BY author')
    notes = []
    for i in sqlalchemyObj:
        notes.append(i)
    dataset=[]
    dict={}
    for i in notes:
        dict['title'] = i[0]
        dict['note_body'] = i[1]
        dict['date_created'] = i[2]
        dict['author'] = i[3]
        dataset.append(dict.copy())
    return render_template('forum.html', dataset=dataset)

@app.route('/new-post', methods=['GET','POST'])
@login_required
def new_post():
    form = NewNoteForm()
    if form.validate_on_submit():
        new_post = Note(title=form.title.data, note_body=form.note_body.data, author=current_user.username)
        db.session.add(new_post)
        db.session.commit()
        return redirect(url_for('note'))
    return render_template('new_post.html',form=form)

@app.route('/event')
def event():
    sqlalchemyObj = db.engine.execute('select * from event ORDER BY Name')
    activity = []
    for i in sqlalchemyObj:
        activity.append(i)
    dataset = []
    dict={}
    for i in activity:
        dict['Name'] = i[0]
        dict['Lat'] = i[1]
        dict['Lng'] = i[2]
        dict['Url'] = i[3]
        dict['Img_url'] = i[4]
        dict['Desc'] = i[5]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    return render_template ('event.html', dataset=dataset)

@app.route('/transport')
def transport():
    return render_template ('transport.html')

@app.route('/book')
@login_required
def book():

    sqlalchemyObj = db.engine.execute('select * from book ORDER BY name')
    book = []
    for i in sqlalchemyObj:
        book.append(i)
    dataset = []
    dict={}
    for i in book:
        dict['Name'] = i[0]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)

    return render_template ('book.html', dataset=dataset)

class food(db.Model):
    __tablename__ = 'food'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    name = db.Column(db.String(25))
    Lat = db.Column(db.Float)
    Lng = db.Column(db.Float)
    url = db.Column(db.Text)
    img_url = db.Column(db.Text)
    img_url2 = db.Column(db.Text)
    img_url3 = db.Column(db.Text)
    Desc = db.Column(db.Text)
    details = db.Column(db.Text)
    opening_hour = db.Column(db.Text)
    recommendation = db.Column(db.Text)


@app.route("/food<food_id>",methods=['GET','POST'])
def foodinfo(food_id):
    item = food.query.get_or_404(int(food_id))

    form = NewNoteForm()
    if form.validate_on_submit():
        new_post = Note(title=form.title.data, note_body=form.note_body.data, author=current_user.username)
        db.session.add(new_post)
        db.session.commit()

    return render_template('fooddetails.html',item=item,form=form)

class activity(db.Model):
    __tablename__ = 'activity'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    name = db.Column(db.String(25))
    Lat = db.Column(db.Float)
    Lng = db.Column(db.Float)
    url = db.Column(db.Text)
    img_url = db.Column(db.Text)
    Desc = db.Column(db.Text)
    details = db.Column(db.Text)

@app.route("/activity<activity_id>",methods=['GET','POST'])
def actinfo(activity_id):
    item = activity.query.get_or_404(int(activity_id))

    form = NewNoteForm()
    if form.validate_on_submit():
        new_post = Note(title=form.title.data, note_body=form.note_body.data, author=current_user.username)
        db.session.add(new_post)
        db.session.commit()

    return render_template('actdetails.html',item=item,form=form)

if __name__ == '__main__':
    app.run('localhost',9009)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9009/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Dec/2021 22:15:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2021 22:15:55] "GET /leaflet.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2021 22:15:55] "GET /leaflet.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2021 22:15:55] "GET /leaflet.js HTTP/1.1" 404 -
